In [1]:
import AlGDock.Nanopore
from AlGDock.Nanopore import *

# This will run a small molecule
#  self = AlGDock.Nanopore.Simulation(\
#    ligand_tarball='prmtopcrd/ligand.tar.gz', \
#    ligand_database='ligand.db', \
#    forcefield='prmtopcrd/gaff2.dat', \
#    frcmodList=['ligand.frcmod'], \
#    ligand_prmtop='ligand.prmtop', \
#    starting_conf='prmtopcrd/anchor_and_grow_scored.mol2', \
#    grid_LJr='grids/LJr.nc', \
#    ef=1.0E8, \
#    max_trials=10000, \
#    report_interval=100)

# This will run a peptide
self = AlGDock.Nanopore.Simulation(\
  ligand_database='prmtopcrd/YFF_14.db', \
  forcefield='prmtopcrd/parm10.dat', \
  frcmodList=['prmtopcrd/frcmod.ff14SB'], \
  ligand_prmtop='prmtopcrd/YFF_14.prmtop', \
  starting_conf='prmtopcrd/YFF_14.inpcrd', \
  grid_LJr='grids/LJr.nc', \
  ef=1.0E8, \
  max_trials=10000, \
  report_interval=100)

The net charge on the ligand is too low for the electric field to have an effect
  grid type LJa not found
  grid type ELE not found


         /Users/dminh/Installers/AlGDock-0.0.1/Nanopore_Example/prmtopcrd/yff_14.db,
         using first one
/Users/dminh/Installers/AlGDock-0.0.1/Nanopore_Example/prmtopcrd/yff_14.db
/Users/dminh/Installers/AlGDock-0.0.1/Nanopore_Example/prmtopcrd/yff_14.db


In [2]:
    from AlGDock.Integrators.HamiltonianMonteCarlo.HamiltonianMonteCarlo \
      import HamiltonianMonteCarloIntegrator
    sampler = HamiltonianMonteCarloIntegrator(self.universe)
    
    e_o = self.universe.energy()
    T_LOW = 20.
    T_START = 300.
    delta_t = 1.5*MMTK.Units.fs
    T_SERIES = T_LOW*(T_START/T_LOW)**(np.arange(30)/29.)
    for T in T_SERIES:
      attempts_left = 5
      while attempts_left>0:
        random_seed = int(T*10000) + attempts_left + \
          int(self.universe.configuration().array[0][0]*10000) + \
          int(time.time())
        random_seed = random_seed%32767
        (xs, energies, acc, ntrials, delta_t) = \
          sampler(steps = 500, steps_per_trial = 50, T=T,\
                  delta_t=delta_t, random_seed=random_seed)
        if (np.std(energies)>1E-3) and float(acc)/ntrials>0.4:
          attempts_left = 0
        else:
          delta_t *= 0.9
          attempts_left -= 1
    self.universe.normalizePosition()
    e_f = self.universe.energy()

In [3]:
print self.universe.energyTerms()

import AlGDock.IO
IO_dcd = AlGDock.IO.dcd(self.molecule,
  ligand_atom_order = self.molecule.prmtop_atom_order)
IO_dcd.write(self.args['output_dcd'], [self.universe.configuration().array])

{'LJr': 57418.36792153868, 'electrostatic/pair sum': -259.52123013193994, 'electric_field_z': 72.023722362006239, 'OBC': -703.20415620579445, 'cosine dihedral angle': 106.05729956020555, 'harmonic bond': 313.68534601157972, 'electrostatic': -259.52123013193994, 'harmonic bond angle': 201.39175405281833, 'Lennard-Jones': 1171.0321428611971}


In [4]:
self.run()

Simulating motion from 0.60000 to 5.40000
Trial      0, COM: [1.5688700664004985e-16, 2.726845591600866e-16, 9.525282546003027e-17]
Trial    100, COM: [1.5688700664004985e-16, 2.726845591600866e-16, 9.525282546003027e-17]
Trial    200, COM: [0.5569267434036593, 0.06139405365880572, 0.17022364791638678]
Trial    300, COM: [0.5569267434036593, 0.06139405365880572, 0.17022364791638678]
Trial    400, COM: [0.5569267434036593, 0.06139405365880572, 0.17022364791638678]
Trial    500, COM: [0.5569267434036593, 0.06139405365880572, 0.17022364791638678]
Trial    600, COM: [0.5569267434036593, 0.06139405365880572, 0.17022364791638678]
Trial    700, COM: [0.5569267434036593, 0.06139405365880572, 0.17022364791638678]
Trial    800, COM: [0.5569267434036593, 0.06139405365880572, 0.17022364791638678]
Trial    900, COM: [0.5569267434036593, 0.06139405365880572, 0.17022364791638678]
Trial   1000, COM: [0.5569267434036593, 0.06139405365880572, 0.17022364791638678]
Trial   1100, COM: [0.5569267434036593, 

KeyboardInterrupt: 

In [5]:
self.view_trajectory()